In [22]:
"""
python_arXiv_parsing_example.py

https://arxiv.org/help/api/examples/python_arXiv_parsing_example.txt

This sample script illustrates a basic arXiv api call
followed by parsing of the results using the 
feedparser python module.

Please see the documentation at 
http://export.arxiv.org/api_help/docs/user-manual.html
for more information, or email the arXiv api 
mailing list at arxiv-api@googlegroups.com.

urllib is included in the standard python library.
feedparser can be downloaded from http://feedparser.org/ .

Author: Julius B. Lucks

This is free software.  Feel free to do what you want
with it, but please play nice with the arXiv API!
"""
import urllib.request
import feedparser

# Base api query url
base_url = 'http://export.arxiv.org/api/query?';

# Search parameters
search_query = 'all:zombie' # search for electron in all fields
start = 0                     # retreive the first 5 results
max_results = 15

query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                     start,
                                                     max_results)

# Opensearch metadata such as totalResults, startIndex, 
# and itemsPerPage live in the opensearch namespase.
# Some entry metadata lives in the arXiv namespace.
# This is a hack to expose both of these namespaces in
# feedparser v4.1
feedparser._FeedParserMixin.namespaces['http://a9.com/-/spec/opensearch/1.1/'] = 'opensearch'
feedparser._FeedParserMixin.namespaces['http://arxiv.org/schemas/atom'] = 'arxiv'

# perform a GET request using the base_url and query
response = urllib.request.urlopen(base_url+query).read()

# parse the response using feedparser
feed = feedparser.parse(response)

# print out feed information
print('Feed title: %s' % feed.feed.title)
print('Feed last updated: %s' % feed.feed.updated)

# print opensearch metadata
print('totalResults for this query: %s' % feed.feed.opensearch_totalresults)
print('itemsPerPage for this query: %s' % feed.feed.opensearch_itemsperpage)
print('startIndex for this query: %s'   % feed.feed.opensearch_startindex)

# Run through each entry, and print out information
for entry in feed.entries:
    print('e-print metadata')
    print('arxiv-id: %s' % entry.id.split('/abs/')[-1])
    print('Published: %s' % entry.published)
    print('Title:  %s' % entry.title)
    
    # feedparser v4.1 only grabs the first author
    author_string = entry.author
    
    # grab the affiliation in <arxiv:affiliation> if present
    # - this will only grab the first affiliation encountered
    #   (the first affiliation for the first author)
    # Please email the list with a way to get all of this information!
    try:
        author_string += ' (%s)' % entry.arxiv_affiliation
    except AttributeError:
        pass
    
    print('Last Author:  %s' % author_string)
    
    # feedparser v5.0.1 correctly handles multiple authors, print them all
    try:
        print('Authors:  %s' % ', ').join(author.name for author in entry.authors)
    except AttributeError:
        pass

    # get the links to the abs page and pdf for this e-print
    for link in entry.links:
        if link.rel == 'alternate':
            print('abs page link: %s' % link.href)
        elif link.title == 'pdf':
            print('pdf link: %s' % link.href)
    
    # The journal reference, comments and primary_category sections live under 
    # the arxiv namespace
    try:
        journal_ref = entry.arxiv_journal_ref
    except AttributeError:
        journal_ref = 'No journal ref found'
    print('Journal reference: %s' % journal_ref)
    
    try:
        comment = entry.arxiv_comment
    except AttributeError:
        comment = 'No comment found'
    print('Comments: %s' % comment)
    
    # Since the <arxiv:primary_category> element has no data, only
    # attributes, feedparser does not store anything inside
    # entry.arxiv_primary_category
    # This is a dirty hack to get the primary_category, just take the
    # first element in entry.tags.  If anyone knows a better way to do
    # this, please email the list!
    print('Primary Category: %s' % entry.tags[0]['term'])
    
    # Lets get all the categories
    all_categories = [t['term'] for t in entry.tags]
    print('All Categories: %s' % (', ').join(all_categories))
    
    # The abstract is in the <summary> element
    print('Abstract: %s' %  entry.summary)

Feed title: ArXiv Query: search_query=all:zombie&id_list=&start=0&max_results=15
Feed last updated: 2017-10-22T00:00:00-04:00
totalResults for this query: 33
itemsPerPage for this query: 15
startIndex for this query: 0
e-print metadata
arxiv-id: 1503.08810v1
Published: 2015-03-30T19:33:25Z
Title:  A probabilistic version of the game of Zombies and Survivors on graphs
Last Author:  Paweł Prałat
Authors:  , 
abs page link: http://arxiv.org/abs/1503.08810v1
pdf link: http://arxiv.org/pdf/1503.08810v1
Journal reference: No journal ref found
Comments: No comment found
Primary Category: cs.DM
All Categories: cs.DM, (Primary) 05C57, (Secondary) 05C80
Abstract: We consider a new probabilistic graph searching game played on graphs,
inspired by the familiar game of Cops and Robbers. In Zombies and Survivors, a
set of zombies attempts to eat a lone survivor loose on a given graph. The
zombies randomly choose their initial location, and during the course of the
game, move directly toward the survi

**refs**
- [arxiv API](https://arxiv.org/help/api/index): [examples](https://arxiv.org/help/api/user-manual#detailed_examples)